# Binary BERT Classifier: Mental Health Detection
This notebook trains a BERT model to classify statements as **Normal (1)** or **Distress (0)**.

In [1]:
# Install dependencies
!pip install transformers datasets evaluate scikit-learn pandas tqdm --quiet

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Imports
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm

# Set seed and device
torch.manual_seed(42)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cuda


In [3]:
# Load dataset
df = pd.read_csv('../data/sentiment.csv')  # Change to your actual file path
df = df[['statement', 'status']].dropna()
df['statement'] = df['statement'].str.strip()
df = df[df['statement'] != '']

# Relabel for binary classification: 1 = Normal, 0 = Distress
df['label'] = df['status'].apply(lambda x: 1 if x.strip().lower() == 'normal' else 0)
print(df['label'].value_counts())

label
0    36338
1    16343
Name: count, dtype: int64


In [4]:
# Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['statement'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})

In [5]:
# Tokenization
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 10537/10537 [00:00<00:00, 12676.21 examples/s]


In [6]:
# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Define recall metric
recall_metric = evaluate.load('recall')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    recall = recall_metric.compute(predictions=predictions, references=labels, average='binary', pos_label=0)
    return {'recall_class_0': recall['recall']}

In [8]:
# Training setup
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True
)

In [9]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train
trainer.train()

Epoch,Training Loss,Validation Loss,Recall Class 0
1,0.047600,0.119032,0.985828
2,0.045900,0.107055,0.982526
3,0.060700,0.196035,0.986929
4,0.000300,0.181705,0.982939


TrainOutput(global_step=10536, training_loss=0.054332927953501794, metrics={'train_runtime': 1354.8632, 'train_samples_per_second': 124.423, 'train_steps_per_second': 7.776, 'total_flos': 1.108855231709184e+16, 'train_loss': 0.054332927953501794, 'epoch': 4.0})